In [234]:
import pulp
import pandas as pd
import matplotlib.pyplot as plt
from pulp import *
import plotly.express as px
import pandas as pd
import numpy as np
import statsmodels.stats.api as sms

In [2]:
def my_process_dataframe_timeline(args):
    """
    Massage input for bar traces for px.timeline()
    """
    print("my method")
    args["is_timeline"] = True
    if args["x_start"] is None or args["x_end"] is None:
        raise ValueError("Both x_start and x_end are required")

    x_start = args["data_frame"][args["x_start"]]
    x_end = args["data_frame"][args["x_end"]]

    # note that we are not adding any columns to the data frame here, so no risk of overwrite
    args["data_frame"][args["x_end"]] = (x_end - x_start)
    args["x"] = args["x_end"]
    del args["x_end"]
    args["base"] = args["x_start"]
    del args["x_start"]
    return args
px._core.process_dataframe_timeline = my_process_dataframe_timeline

In [3]:
import plotly.io as pio
pio.renderers.default='notebook'

### 1. MODA

模型列式：  
minimun energy: $$Min: c_{max} $$
minimun penalty: $$Min: TEC $$


### mean and variance

In [195]:
np.random.seed(87)
pi1 = np.random.normal(6, 1, 15)
pi2 = np.random.normal(8, 0.5, 15)
pi3 = np.random.normal(10, 2 ,15)

In [209]:
p_dict = {}

p_dict[(1,1)], p_dict[(1,2)], p_dict[(1,3)], p_dict[(1,4)], p_dict[(1,5)], p_dict[(1,6)] =  3,6,4,7,8,2
p_dict[(2,1)], p_dict[(2,2)], p_dict[(2,3)], p_dict[(2,4)], p_dict[(2,5)], p_dict[(2,6)] =  5,8,4,5,2,2
p_dict[(3,1)], p_dict[(3,2)], p_dict[(3,3)], p_dict[(3,4)], p_dict[(3,5)], p_dict[(3,6)] =  7,8,4,4,3,2

In [210]:
p_var_dict = {}

p_var_dict[(1,1)], p_var_dict[(1,2)], p_var_dict[(1,3)], p_var_dict[(1,4)], p_var_dict[(1,5)], p_var_dict[(1,6)] = 0.3,0.3,0.2,0.2,1,1
p_var_dict[(2,1)], p_var_dict[(2,2)], p_var_dict[(2,3)], p_var_dict[(2,4)], p_var_dict[(2,5)], p_var_dict[(2,6)] = 0.1,0.1,0.5,0.5,0.3,0.3 
p_var_dict[(3,1)], p_var_dict[(3,2)], p_var_dict[(3,3)], p_var_dict[(3,4)], p_var_dict[(3,5)], p_var_dict[(3,6)] = 0.7,0.7,1,1,0.1,0.1

In [218]:
np.random.seed(87)
p_rand_dict = {}

for i in range(1,4,1):
    for j in range(1,7,1):
        p_rand_dict[(i,j)] = list(np.random.normal(p_dict[(i,j)],p_var_dict[(i,j)],15))

In [258]:
def op_lowbound(obj,index):
    new_v_list = [1,1,1]
    new_lam_list = [1,1,1]
    new_o_list = [0,0,0]    
    M = 3 # machine_num
    N = 6 # job_num

    p_dict = {}

    for i in range(1,4,1):
        for j in range(1,7,1):
            p_dict[(i,j)] = p_rand_dict[(i,j)][index]

    pi_dict = {}
    pi_dict[1], pi_dict[2], pi_dict[3] = pi1[index],pi2[index],pi3[index]

    v_dict = {}
    lam_dict = {}
    v_dict[1], v_dict[2], v_dict[3] = new_v_list[0],new_v_list[1],new_v_list[2]
    lam_dict[1], lam_dict[2], lam_dict[3] = new_lam_list[0],new_lam_list[1],new_lam_list[2]
    B = 10**6
    time_list = []

    for i in range(3):
        time_list.append(new_o_list[i])

    x_dict = {}
    for i in range(1,M+1,1):
        for j in range(N+1):
            for k in range(N+1):
                if (j != k):
                    x_dict[(i,j,k)] = LpVariable("x"+str((i,j,k)),lowBound=0, upBound=1,cat="Binary")
    c_dict = {}
    for j in range(N+1):
        c_dict[(j)] = LpVariable("c"+str(j),lowBound=0,cat="Continuous")                

    o_dict = {}
    for i in range(M+1):
        o_dict[(i)] = LpVariable("o"+str(i),lowBound=0,cat="Continuous")

    c_max = LpVariable("c_max",lowBound=0,cat="Continuous")
    TEC = LpVariable("TEC",lowBound=0,cat="Continuous")

    problem = pulp.LpProblem("Mim energy",LpMinimize)
    
    ################################################
    if obj == 1:
        problem += c_max 
    elif obj == 2:
        problem += TEC
    elif obj == 1.5:
        problem += c_max
        problem += TEC <= (op_lowbound(1,index)[1] + op_lowbound(2,index)[1])/2
    ################################################
    
    for k in range(1,N+1,1):
        con_list = []
        for i in range(1,M+1,1):
            for j in range(N+1):
                    if j != k:
                        con_list.append(x_dict[(i,j,k)])
        problem += lpSum(con_list) == 1

    for j in range(1,N+1,1):
        con_list = []
        for i in range(1,M+1,1):
            for k in range(1,N+1,1):
                    if j != k:
                        con_list.append(x_dict[(i,j,k)])
        problem += lpSum(con_list) <= 1

    for i in range(1,M+1,1):
        con_list = []
        for k in range(1,N+1,1):
                con_list.append(x_dict[(i,0,k)])
        problem += lpSum(con_list) <= 1

    for j in range(1,N+1,1):
        for i in range(1,M+1,1):
            sep_one_list = []
            sep_two_list = []
            for k in range(N+1):
                    if j != k:
                        sep_one_list.append(x_dict[(i,j,k)])
            for h in range(N+1):
                    if j != h:
                        sep_two_list.append(x_dict[(i,h,j)])   
            problem += lpSum(sep_one_list) - lpSum(sep_two_list) == 0

    for i in range(1,M+1,1):
            for j in range(N+1):
                for k in range(1,N+1,+1):
                    if j != k:
                        problem += c_dict[k] - c_dict[j] + B*(1-x_dict[(i,j,k)]) >= (p_dict[(i,k)] / v_dict[i]) 

    problem += c_dict[0] == 0

    ####################################################################################
    ## new_o
    for i in range(1,M+1,1):
        con_list = []
        for j in range(N+1):
            for k in range(1,N+1,1):
                    if j != k:
                        con_list.append([ (p_dict[(i,k)]/v_dict[i]) * x_dict[(i,j,k)] ])
        problem += lpSum(con_list) + time_list[i-1] == o_dict[i]
    ####################################################################################

    for i in range(1,M+1,1):
        problem += c_max >= o_dict[i]

    con_list = []
    for i in range(1,M+1,1):
        for j in range(N+1):
            for k in range(1,N+1,1):
                    if j != k:
                        con_list.append([ (lam_dict[i]*pi_dict[i]*p_dict[(i,k)]/v_dict[i]) * x_dict[(i,j,k)]  ])
    problem += lpSum(con_list) <= TEC


    con_list = []
    for i in range(1,M+1,1):
        for j in range(N+1):
            for k in range(1,N+1,1):
                    if j != k:
                        con_list.append([ (lam_dict[i]*pi_dict[i]*p_dict[(i,k)]/v_dict[i]) * x_dict[(i,j,k)]  ])
    problem += lpSum(con_list) <= TEC    
    solution = problem.solve()
    ####################################################################################
    last_ans = []
    for v in problem.variables():
        if "x" in v.name and "c" not in v.name:
            last_ans.append(v.varValue)
    ####################################################################################
    return [value(c_max),value(TEC),last_ans]

### obj c_max =1 TEC = 2 epision = 1.5

#### c_max low bound

In [259]:
ans = {1:[],1.5:[],2:[]}

In [260]:
c_max_list = []
tec_list = []
for i in range(15):
    result = op_lowbound(1,i)
    print(result[0:2])
    c_max_list.append(result[0])
    tec_list.append(result[1])
    ans[1].append(result[2])

[8.837651, 167.53289]
[8.1653096, 161.52105]
[8.4218256, 208.02584]
[7.0967317, 167.12445]
[8.6455381, 200.5338]
[8.871934, 171.13097]
[6.372035, 138.11094]
[8.198506, 187.60062]
[8.6591398, 153.81994]
[8.6768804, 216.13277]
[7.9708549, 176.16817]
[7.7754621, 188.72655]
[9.1924539, 189.96358]
[8.4284595, 179.34059]
[8.0521228, 155.34008]


In [261]:
sms.DescrStatsW(c_max_list).tconfint_mean(0.1) , sms.DescrStatsW(tec_list).tconfint_mean(0.1)

((7.893992566018291, 8.554661353981711),
 (167.64393806401029, 187.16569393598974))

#### TEC low bound

In [262]:
c_max_list = []
tec_list = []
for i in range(15):
    result = op_lowbound(2,i)
    print(result[0:2])
    c_max_list.append(result[0])
    tec_list.append(result[1])
    ans[2].append(result[2])

[14.589248, 146.57558]
[14.092113, 130.45904]
[13.067809, 145.69177]
[14.149224, 126.9123]
[13.106829, 168.24994]
[12.997594, 156.24355]
[12.449386, 102.65894]
[13.246195, 153.64764]
[20.214645, 128.25874]
[14.856376, 166.20896]
[20.083039, 129.69661]
[21.85752, 119.50556]
[13.622141, 167.6956]
[14.316597, 130.20949]
[21.267915, 123.80283]


#### epision low bound

In [263]:
sms.DescrStatsW(c_max_list).tconfint_mean(0.1) , sms.DescrStatsW(tec_list).tconfint_mean(0.1)

((14.061599884254367, 17.127284249078965),
 (130.74384389319312, 148.69836277347355))

In [264]:
c_max_list = []
tec_list = []
for i in range(15):
    result = op_lowbound(1.5,i)
    print(result[0:2])
    c_max_list.append(result[0])
    tec_list.append(result[1])
    ans[1.5].append(result[2])

[10.499364, 154.56985]
[8.4896543, 140.71432]
[8.8601395, 166.10245]
[8.5061504, 134.4282]
[9.1727544, 176.52183]
[10.945954, 162.68492]
[8.6440866, 109.20678]
[9.1141865, 156.22789]
[12.304799, 140.33334]
[10.786929, 187.1093]
[9.3417421, 150.12331]
[8.596588, 144.84875]
[9.9025834, 171.86628]
[8.4376396, 146.61343]
[8.4769551, 131.85497]


In [265]:
sms.DescrStatsW(c_max_list).tconfint_mean(0.1) , sms.DescrStatsW(tec_list).tconfint_mean(0.1)

((8.939595277439096, 10.004341509227569),
 (142.569290138963, 160.52479252770362))

In [312]:
def op_30_up(obj,index):
    sample_list = []
    for sample in range(30):
        M = 3 # machine_num
        N = 6 # job_num

        p_dict = {}

        for i in range(1,4,1):
            for j in range(1,7,1):
                p_dict[(i,j)] = p_rand_dict[(i,j)][sample]

        pi_dict = {}
        pi_dict[1], pi_dict[2], pi_dict[3] = pi1[sample],pi2[sample],pi3[sample]

        v_dict = {}
        lam_dict = {}
        v_dict[1], v_dict[2], v_dict[3] = new_v_list[0],new_v_list[1],new_v_list[2]
        lam_dict[1], lam_dict[2], lam_dict[3] = new_lam_list[0],new_lam_list[1],new_lam_list[2]
        B = 10**6
        time_list = []

        for i in range(3):
            time_list.append(new_o_list[i])

        x_dict = {}
        for i in range(1,M+1,1):
            for j in range(N+1):
                for k in range(N+1):
                    if (j != k):
                        x_dict[(i,j,k)] = LpVariable("x"+str((i,j,k)),lowBound=0, upBound=1,cat="Binary")
        c_dict = {}
        for j in range(N+1):
            c_dict[(j)] = LpVariable("c"+str(j),lowBound=0,cat="Continuous")                

        o_dict = {}
        for i in range(M+1):
            o_dict[(i)] = LpVariable("o"+str(i),lowBound=0,cat="Continuous")

        c_max = LpVariable("c_max",lowBound=0,cat="Continuous")
        TEC = LpVariable("TEC",lowBound=0,cat="Continuous")

        problem = pulp.LpProblem("Mim energy",LpMinimize)

        if obj == 1:
            problem += c_max 
        elif obj ==2:
            problem += TEC

        for k in range(1,N+1,1):
            con_list = []
            for i in range(1,M+1,1):
                for j in range(N+1):
                        if j != k:
                            con_list.append(x_dict[(i,j,k)])
            problem += lpSum(con_list) == 1

        for j in range(1,N+1,1):
            con_list = []
            for i in range(1,M+1,1):
                for k in range(1,N+1,1):
                        if j != k:
                            con_list.append(x_dict[(i,j,k)])
            problem += lpSum(con_list) <= 1

        for i in range(1,M+1,1):
            con_list = []
            for k in range(1,N+1,1):
                    con_list.append(x_dict[(i,0,k)])
            problem += lpSum(con_list) <= 1

        for j in range(1,N+1,1):
            for i in range(1,M+1,1):
                sep_one_list = []
                sep_two_list = []
                for k in range(N+1):
                        if j != k:
                            sep_one_list.append(x_dict[(i,j,k)])
                for h in range(N+1):
                        if j != h:
                            sep_two_list.append(x_dict[(i,h,j)])   
                problem += lpSum(sep_one_list) - lpSum(sep_two_list) == 0

        for i in range(1,M+1,1):
                for j in range(N+1):
                    for k in range(1,N+1,+1):
                        if j != k:
                            problem += c_dict[k] - c_dict[j] + B*(1-x_dict[(i,j,k)]) >= (p_dict[(i,k)] / v_dict[i]) 

        problem += c_dict[0] == 0

        ####################################################################################
        ## new_o
        for i in range(1,M+1,1):
            con_list = []
            for j in range(N+1):
                for k in range(1,N+1,1):
                        if j != k:
                            con_list.append([ (p_dict[(i,k)]/v_dict[i]) * x_dict[(i,j,k)] ])
            problem += lpSum(con_list) + time_list[i-1] == o_dict[i]
        ####################################################################################

        for i in range(1,M+1,1):
            problem += c_max >= o_dict[i]

        con_list = []
        for i in range(1,M+1,1):
            for j in range(N+1):
                for k in range(1,N+1,1):
                        if j != k:
                            con_list.append([ (lam_dict[i]*pi_dict[i]*p_dict[(i,k)]/v_dict[i]) * x_dict[(i,j,k)]  ])
        problem += lpSum(con_list) <= TEC


        con_list = []
        for i in range(1,M+1,1):
            for j in range(N+1):
                for k in range(1,N+1,1):
                        if j != k:
                            con_list.append([ (lam_dict[i]*pi_dict[i]*p_dict[(i,k)]/v_dict[i]) * x_dict[(i,j,k)]  ])
        problem += lpSum(con_list) <= TEC
        ##################################################################################

        for i in range(len(ans[obj][index])):
            problem += x_dict[list(x_dict.keys())[i]] == ans[obj][index][i]

        ##################################################################################    
        solution = problem.solve()
        sample_list.append([value(c_max),value(TEC)])
    return [np.mean(np.array(sample_list).T[0]), np.mean(np.array(sample_list).T[1])]

#### c_max up bound

In [321]:
c_max_list = []
tec_list = []
for i in range(15):
    
    np.random.seed(i)
    pi1 = np.random.normal(6, 1, 30)
    pi2 = np.random.normal(8, 0.5, 30)
    pi3 = np.random.normal(10, 2 ,30)
    p_dict = {}

    p_dict[(1,1)], p_dict[(1,2)], p_dict[(1,3)], p_dict[(1,4)], p_dict[(1,5)], p_dict[(1,6)] =  3,6,4,7,8,2
    p_dict[(2,1)], p_dict[(2,2)], p_dict[(2,3)], p_dict[(2,4)], p_dict[(2,5)], p_dict[(2,6)] =  5,8,4,5,2,2
    p_dict[(3,1)], p_dict[(3,2)], p_dict[(3,3)], p_dict[(3,4)], p_dict[(3,5)], p_dict[(3,6)] =  7,8,4,4,3,2

    p_var_dict = {}

    p_var_dict[(1,1)], p_var_dict[(1,2)], p_var_dict[(1,3)], p_var_dict[(1,4)], p_var_dict[(1,5)], p_var_dict[(1,6)] = 0.3,0.3,0.2,0.2,1,1
    p_var_dict[(2,1)], p_var_dict[(2,2)], p_var_dict[(2,3)], p_var_dict[(2,4)], p_var_dict[(2,5)], p_var_dict[(2,6)] = 0.1,0.1,0.5,0.5,0.3,0.3 
    p_var_dict[(3,1)], p_var_dict[(3,2)], p_var_dict[(3,3)], p_var_dict[(3,4)], p_var_dict[(3,5)], p_var_dict[(3,6)] = 0.7,0.7,1,1,0.1,0.1

    p_rand_dict = {}

    for i in range(1,4,1):
        for j in range(1,7,1):
            p_rand_dict[(i,j)] = list(np.random.normal(p_dict[(i,j)],p_var_dict[(i,j)],30))            
    res = op_30_up(1,i)
    print([res[0],res[1]])
    c_max_list.append(res[0])
    tec_list.append(res[1])

[8.33289531, 180.5957973333333]
[8.931296693333334, 188.92630000000003]
[8.38159151, 173.41215333333332]
[8.904231273333332, 184.03932633333335]
[8.992159446666667, 192.571756]
[8.878837983333334, 186.88045033333336]
[8.754328730000001, 187.38050966666665]
[8.413357976666667, 179.765496]
[9.073236756666665, 191.01200866666667]
[8.704791236666667, 183.99024133333333]
[8.48311537, 180.74583900000005]
[8.758872123333333, 188.08323166666668]
[8.434597819999999, 177.91816466666668]
[8.406296430000001, 179.33349533333333]
[8.602708856666668, 176.88597666666666]


In [322]:
sms.DescrStatsW(c_max_list).tconfint_mean(0.1) , sms.DescrStatsW(tec_list).tconfint_mean(0.1)

((8.55636814825748, 8.783940853964744),
 (180.88619832869915, 185.9859011824119))

#### TEC low bound

In [323]:
c_max_list = []
tec_list = []
for i in range(15):
    
    np.random.seed(i)
    pi1 = np.random.normal(6, 1, 30)
    pi2 = np.random.normal(8, 0.5, 30)
    pi3 = np.random.normal(10, 2 ,30)
    p_dict = {}

    p_dict[(1,1)], p_dict[(1,2)], p_dict[(1,3)], p_dict[(1,4)], p_dict[(1,5)], p_dict[(1,6)] =  3,6,4,7,8,2
    p_dict[(2,1)], p_dict[(2,2)], p_dict[(2,3)], p_dict[(2,4)], p_dict[(2,5)], p_dict[(2,6)] =  5,8,4,5,2,2
    p_dict[(3,1)], p_dict[(3,2)], p_dict[(3,3)], p_dict[(3,4)], p_dict[(3,5)], p_dict[(3,6)] =  7,8,4,4,3,2

    p_var_dict = {}

    p_var_dict[(1,1)], p_var_dict[(1,2)], p_var_dict[(1,3)], p_var_dict[(1,4)], p_var_dict[(1,5)], p_var_dict[(1,6)] = 0.3,0.3,0.2,0.2,1,1
    p_var_dict[(2,1)], p_var_dict[(2,2)], p_var_dict[(2,3)], p_var_dict[(2,4)], p_var_dict[(2,5)], p_var_dict[(2,6)] = 0.1,0.1,0.5,0.5,0.3,0.3 
    p_var_dict[(3,1)], p_var_dict[(3,2)], p_var_dict[(3,3)], p_var_dict[(3,4)], p_var_dict[(3,5)], p_var_dict[(3,6)] = 0.7,0.7,1,1,0.1,0.1

    p_rand_dict = {}

    for i in range(1,4,1):
        for j in range(1,7,1):
            p_rand_dict[(i,j)] = list(np.random.normal(p_dict[(i,j)],p_var_dict[(i,j)],30))            
    res = op_30_up(2,i)
    print([res[0],res[1]])
    c_max_list.append(res[0])
    tec_list.append(res[1])

[14.8647815, 150.13547123333333]
[15.040729599999999, 145.19483633333328]
[14.815086999999998, 141.78344433333334]
[15.423149066666666, 145.14189566666667]
[15.124378466666666, 147.07403066666663]
[15.1093323, 147.72413699999998]
[15.170168266666668, 150.94013933333335]
[14.961622333333334, 143.71935166666668]
[14.8988512, 146.01309233333333]
[15.177124333333333, 147.502441]
[14.939084733333333, 148.4486646666667]
[15.323796899999998, 147.56168066666666]
[14.497907666666666, 138.466106]
[14.776588833333332, 147.71972233333332]
[14.654159899999996, 143.41172166666666]


In [324]:
sms.DescrStatsW(c_max_list).tconfint_mean(0.1) , sms.DescrStatsW(tec_list).tconfint_mean(0.1)

((14.872437139300938, 15.097797807365726),
 (144.58186163789938, 147.52970301543397))

#### epision low bound

In [325]:
c_max_list = []
tec_list = []
for i in range(15):
    
    np.random.seed(i)
    pi1 = np.random.normal(6, 1, 30)
    pi2 = np.random.normal(8, 0.5, 30)
    pi3 = np.random.normal(10, 2 ,30)
    p_dict = {}

    p_dict[(1,1)], p_dict[(1,2)], p_dict[(1,3)], p_dict[(1,4)], p_dict[(1,5)], p_dict[(1,6)] =  3,6,4,7,8,2
    p_dict[(2,1)], p_dict[(2,2)], p_dict[(2,3)], p_dict[(2,4)], p_dict[(2,5)], p_dict[(2,6)] =  5,8,4,5,2,2
    p_dict[(3,1)], p_dict[(3,2)], p_dict[(3,3)], p_dict[(3,4)], p_dict[(3,5)], p_dict[(3,6)] =  7,8,4,4,3,2

    p_var_dict = {}

    p_var_dict[(1,1)], p_var_dict[(1,2)], p_var_dict[(1,3)], p_var_dict[(1,4)], p_var_dict[(1,5)], p_var_dict[(1,6)] = 0.3,0.3,0.2,0.2,1,1
    p_var_dict[(2,1)], p_var_dict[(2,2)], p_var_dict[(2,3)], p_var_dict[(2,4)], p_var_dict[(2,5)], p_var_dict[(2,6)] = 0.1,0.1,0.5,0.5,0.3,0.3 
    p_var_dict[(3,1)], p_var_dict[(3,2)], p_var_dict[(3,3)], p_var_dict[(3,4)], p_var_dict[(3,5)], p_var_dict[(3,6)] = 0.7,0.7,1,1,0.1,0.1

    p_rand_dict = {}

    for i in range(1,4,1):
        for j in range(1,7,1):
            p_rand_dict[(i,j)] = list(np.random.normal(p_dict[(i,j)],p_var_dict[(i,j)],30))            
    res = op_30_up(1.5,i)
    print([res[0],res[1]])
    c_max_list.append(res[0])
    tec_list.append(res[1])

[9.371022183333334, 166.44512833333334]
[9.386800923333334, 170.40625799999992]
[9.524969166666667, 160.86705266666667]
[9.255635763333336, 164.253493]
[9.284719576666667, 169.8810133333333]
[9.414114846666667, 169.7001213333333]
[9.204572469999999, 168.75175866666672]
[9.170219536666668, 161.04673366666665]
[9.279828856666667, 171.49219300000001]
[9.35620583, 167.4254013333333]
[9.315931316666667, 162.96470933333336]
[9.327117676666665, 164.19026600000004]
[9.200412553333333, 163.97835466666666]
[9.1692316, 162.70802733333332]
[9.234546156666665, 160.960944]


In [326]:
sms.DescrStatsW(c_max_list).tconfint_mean(0.1) , sms.DescrStatsW(tec_list).tconfint_mean(0.1)

((9.254176198172178, 9.345200929383378),
 (163.98181452692404, 167.36104609529815))